In [1]:
import numpy as np 
import pandas as pd
from IPython.display import display


In [2]:
match=pd.read_csv('..//Data/matches.csv')
delivery=pd.read_csv('../Data/deliveries.csv')

display(match.head())
display(match.shape)

display(delivery.head())
display(delivery.shape)

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


(756, 18)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


(179078, 21)

In [3]:
total_score_df=delivery.groupby(['match_id','inning']).sum()['total_runs'].reset_index()
# this gives total score of both first and second innings but we want secong inning so 
total_score_df=total_score_df[total_score_df['inning']==1]
# ---display(total_score_df)

match_df=match.merge(total_score_df[['match_id','total_runs']],left_on='id',right_on='match_id')
# ---display(match_df)
display(match_df.shape)

# this data is from 2008 so some teams are changed and some are not playing so we will remove them or replace them 
match['team1'].unique()

teams=['Sunrisers Hyderabad',
       'Mumbai Indians',
       'Royal Challengers Bengalore',
       'Kolkata Knight Riders',
       'Kings XI Punjab',
       'Chennai Super Kings',
       'Rajasthan Royals',
       'Delhi Capitals'
       ]

match_df['team1']=match_df['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['team2']=match_df['team2'].str.replace('Delhi Daredevils','Delhi Capitals')

match_df['team1']=match_df['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['team2']=match_df['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

# after replacing current team from old team now we have to do one more thing
# in teams column we will remove those who are not playing right now
match_df=match_df[match_df['team1'].isin(teams)]
match_df=match_df[match_df['team2'].isin(teams)]
display(match_df)
display(match_df.shape)

# lets check in how many matches the D/L rule was applied
display(match_df['dl_applied'].value_counts())
# so we will remove all the matches in which D/L was applied because they are not useful for us 
match_df=match_df[match_df["dl_applied"]==0]
match_df=match_df[['match_id','city','winner','total_runs']]
#now we will merge all the data to delivery to get these
delivery_df=match_df.merge(delivery,on='match_id')

# we are building a model that predicts in second innings so we will remove 1st innings from delivery_df
delivery_df=delivery_df[delivery_df['inning']==2]
display(delivery_df.shape)
#to build our model we have the data column that are: batting team,bowling team, city,runs left,balls left, wickets left,total runs ,current rr, required rr, result
""" lets get them """
# finding current score
delivery_df['current_score']=delivery_df.groupby('match_id')['total_runs_y'].cumsum()

# lets move to find runs left : to find this we just simply do total_runs - current_score
delivery_df['runs_left']=delivery_df['total_runs_x'] - delivery_df['current_score']

# lets find out balls left
delivery_df['balls_left']=126-(delivery_df['over']*6+delivery_df['ball'])
#lets find out wickets left
delivery_df['player_dismissed']=delivery_df['player_dismissed'].fillna('0')
delivery_df['player_dismissed']=delivery_df['player_dismissed'].apply(lambda x:x if x=="0" else "1")
delivery_df['player_dismissed']=delivery_df['player_dismissed'].astype('int')
wicket=delivery_df.groupby('match_id')['player_dismissed'].cumsum()
delivery_df['wickets_left']=10-wicket

# lets find out current run rate
delivery_df['current_rr']=(delivery_df['current_score']) / ((120-delivery_df['balls_left'])/6)
# now lets move to required run rate
delivery_df['required_rr']=(delivery_df['runs_left'])/((delivery_df['balls_left'])/6)
# result
def result(row):
    if row['batting_team']==row['winner']:
        return 1
    else:
        return 0
delivery_df['result']=delivery_df.apply(result,axis=1)
display(delivery_df)

final_df=delivery_df[['batting_team','bowling_team','city','runs_left','balls_left',
                     'wickets_left','total_runs_x','current_rr','required_rr','result']]

#shuffling the data for best output of model
final_df=final_df.sample(final_df.shape[0])
display(final_df)
display(final_df.sample())

(756, 20)

,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3,match_id,total_runs
6,7,IPL-2017,Mumbai,09-04-2017,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,4,N Rana,Wankhede Stadium,Nitin Menon,CK Nandan,NaN,7,178
9,10,IPL-2017,Mumbai,12-04-2017,Sunrisers Hyderabad,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,4,JJ Bumrah,Wankhede Stadium,Nitin Menon,CK Nandan,NaN,10,158
10,11,IPL-2017,Kolkata,13-04-2017,Kings XI Punjab,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,8,SP Narine,Eden Gardens,A Deshmukh,NJ Llong,NaN,11,170
13,14,IPL-2017,Kolkata,15-04-2017,Kolkata Knight Riders,Sunrisers Hyderabad,Sunrisers Hyderabad,field,normal,0,Kolkata Knight Riders,17,0,RV Uthappa,Eden Gardens,AY Dandekar,NJ Llong,NaN,14,172
14,15,IPL-2017,Delhi,15-04-2017,Delhi Capitals,Kings XI Punjab,Delhi Daredevils,bat,normal,0,Delhi Daredevils,51,0,CJ Anderson,Feroz Shah Kotla,YC Barde,Nitin Menon,NaN,15,188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,11347,IPL-2019,Mumbai,05-05-2019,Kolkata Knight Riders,Mumbai Indians,Mumbai Indians,field,normal,0,Mumbai Indians,0,9,HH Pandya,Wankhede Stadium,Nanda Kishore,O Nandan,S Ravi,11347,143
752,11412,IPL-2019,Chennai,07-05-2019,Chennai Super Kings,Mumbai Indians,Chennai Super Kings,bat,normal,0,Mumbai Indians,0,6,AS Yadav,M. A. Chidambaram Stadium,Nigel Llong,Nitin Menon,Ian Gould,11412,136
753,11413,IPL-2019,Visakhapatnam,08-05-2019,Sunrisers Hyderabad,Delhi Capitals,Delhi Capitals,field,normal,0,Delhi Capitals,0,2,RR Pant,ACA-VDCA Stadium,NaN,NaN,NaN,11413,171
754,11414,IPL-2019,Visakhapatnam,10-05-2019,Delhi Capitals,Chennai Super Kings,Chennai Super Kings,field,normal,0,Chennai Super Kings,0,6,F du Plessis,ACA-VDCA Stadium,Sundaram Ravi,Bruce Oxenford,Chettithody Shamshuddin,11414,155


(477, 20)

dl_applied
0    465
1     12
Name: count, dtype: int64

(54192, 24)

,match_id,city,winner,total_runs_x,inning,batting_team,bowling_team,over,ball,batsman,...,player_dismissed,dismissal_kind,fielder,current_score,runs_left,balls_left,wickets_left,current_rr,required_rr,result
128,7,Mumbai,Mumbai Indians,178,2,Mumbai Indians,Kolkata Knight Riders,1,1,PA Patel,...,0,NaN,NaN,0,178,119,10,0.000000,8.974790,1
129,7,Mumbai,Mumbai Indians,178,2,Mumbai Indians,Kolkata Knight Riders,1,2,PA Patel,...,0,NaN,NaN,0,178,118,10,0.000000,9.050847,1
130,7,Mumbai,Mumbai Indians,178,2,Mumbai Indians,Kolkata Knight Riders,1,3,PA Patel,...,0,NaN,NaN,4,174,117,10,8.000000,8.923077,1
131,7,Mumbai,Mumbai Indians,178,2,Mumbai Indians,Kolkata Knight Riders,1,4,PA Patel,...,0,NaN,NaN,4,174,116,10,6.000000,9.000000,1
132,7,Mumbai,Mumbai Indians,178,2,Mumbai Indians,Kolkata Knight Riders,1,5,PA Patel,...,0,NaN,NaN,5,173,115,10,6.000000,9.026087,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111755,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,2,RA Jadeja,...,0,NaN,NaN,152,0,4,5,7.862069,0.000000,0
111756,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,3,SR Watson,...,0,NaN,NaN,154,-2,3,5,7.897436,-4.000000,0
111757,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,4,SR Watson,...,1,run out,KH Pandya,155,-3,2,4,7.881356,-9.000000,0
111758,11415,Hyderabad,Mumbai Indians,152,2,Chennai Super Kings,Mumbai Indians,20,5,SN Thakur,...,0,NaN,NaN,157,-5,1,4,7.915966,-30.000000,0


,batting_team,bowling_team,city,runs_left,balls_left,wickets_left,total_runs_x,current_rr,required_rr,result
86806,Mumbai Indians,Delhi Daredevils,Delhi,67,50,8,164,8.314286,8.040000,0
82576,Mumbai Indians,Chennai Super Kings,Chennai,43,22,7,158,7.040816,11.727273,1
110463,Kings XI Punjab,Chennai Super Kings,Mohali,120,98,10,172,14.181818,7.346939,1
48994,Mumbai Indians,Chennai Super Kings,Mumbai,100,61,9,173,7.423729,9.836066,1
107370,Delhi Capitals,Kings XI Punjab,Delhi,109,86,9,168,10.411765,7.604651,1
...,...,...,...,...,...,...,...,...,...,...
69952,Chennai Super Kings,Mumbai Indians,Mumbai,18,14,6,157,7.867925,7.714286,1
71851,Rajasthan Royals,Mumbai Indians,Ahmedabad,109,58,5,178,6.677419,11.275862,0
110235,Delhi Capitals,Rajasthan Royals,Delhi,101,107,10,124,10.615385,5.663551,1
92857,Sunrisers Hyderabad,Kings XI Punjab,Mohali,10,-1,5,201,9.471074,-60.000000,0


,batting_team,bowling_team,city,runs_left,balls_left,wickets_left,total_runs_x,current_rr,required_rr,result
70380,Mumbai Indians,Sunrisers Hyderabad,Hyderabad,88,62,9,157,7.137931,8.516129,1
